# About this Notebook

Data Wrangling can be defined as the process to:
* gather
* assess
* clean
data in support of data analytics tasks.

This notebook contains the code to extract data related to Master Swim Performances and save it in a local format that supports Data Exploration activities.

## Supported Data Sources

Currently data is only extracted from the Canadian swimming site that collects the results for Masters Swimmers.

In [ ]:
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# Canadian Master Swimming Results Collection 

Starting with the information I know how to access from personal experience.

## Gather - www.swimming.ca
The following code will scrape the available data from the www.swimming.ca website for Masters Swimmers. The code is based on the website structure that exists during the time period Aug - Oct 2020.

The website has a layered approach to finding results.
1. Navigate to the page that has a list of meet-results
2. Filter the list of meet results by **Season**, **Province**, **Month**
3. Select the meet of interest in the filtered list
4. At this point it is possible to see all the results for each club that participated, or the results by event.  I will be collecting all the results by participating club.

I did contact Swim Canada to see if there was an API that would allow me to use an approach other than web scraping to get the results, but so far they have not been able to support my request.

For the parameters **Seasion**, **Province** and **Month**, there are few points to note:
* Season - the swim season in Canada is split over 2 calendar years and runs Sep to Aug.  So the swim season 2019/2020 will be the period 1 Sep 2019 to 31 Aug 2020.  The value passed to the **Season** parameter is the last 2 numbers in the second part of the season identifier.  So to access swim meets from Nov 2019, the season parameter will be 20 ('season'='20')
* Month - the month must be specified as a number from 1 to 12.  It is not possible to have all the months in a single season being available at one time
* Province - It is possible to filter by province/territory or use all provinces. To access an individual province/territory, the provinces/territories are numbered from 1 to 11 in alphabetical order.  So to get only Alberta, 'province'='1' and to get Yukon, 'province'='11'.  The supported Territories are Northwest Territories and Yukon.  For all provinces, no number is provided to province (province = '')

Since I am only interested in Masters results, when looking at the meet list, I will look for MEET TYPE  of Masters.  This will miss some swimmers that compete in non-Masters meets as an Open Master.  I plan to come back and find these results after collecting the Masters Meets

The only URL specified in my code is the URL associated with step 1 above, navigating to the meet-results page.  The subsequent URLs will be recovered from the information available when the various filters are applied.

In [ ]:
# URLs of interest
url_meetlist ='https://www.swimming.ca/en/events-results/meet-results/'

In [ ]:
# start with the 2019/2020 season, noting that the season came to an abrupt end in March 2020 because of Covid19
# using November since I know there is a masters meet in Nov 2019
response=requests.get('https://www.swimming.ca/en/events-results/meet-results/', params={'season':'20', 'province':'', 'month':'11'})
#sanity check that my parameter specification worked as expected
response.url

In [ ]:
#use Beautiful Soup to parse the returned page
meetList_resp = BeautifulSoup(response.text, 'lxml')

In [ ]:
#find the masters swim meets in the response text
meetList = []
for item in meetList_resp.find_all('tr'):
    if item.contents[5].contents[0] == "Masters":
        temp_dict = {}
        temp_dict['meet_date'] = item.contents[0].contents[0].contents[0]
        temp_dict['meet_url'] = item.contents[1].a.attrs['href']
        temp_dict['meet_prov'] = item.contents[2].contents[0]
        temp_dict['meet_host'] = item.contents[3].contents[0].contents[0]
        temp_dict['meet_course'] = item.contents[4].contents[0]
        temp_dict['meet_type'] = item.contents[5].contents[0]
        meetList.append(temp_dict)
        ##used for debug/development
        #print(item.contents[0].contents[0].contents[0])
        #print(item.contents[1].a.attrs['href'])
        #print(item.contents[2].contents[0])
        #print(item.contents[3].contents[0].contents[0])
        #print(item.contents[4].contents[0])
        #print(item.contents[5].contents[0])
print("number of masters meets", len(meetList))

So 8 masters meets were held in Canada in Nov 2019.

Now use the meet_url to get the results for that meet.  The use Beautiful Soup to parse the returned page

In [ ]:
r2_url = meetList[0]['meet_url']
r2 = requests.get(r2_url)
temp_r2 = BeautifulSoup(r2.text, 'lxml')

My next step was to create a list of clubs that participated. I looked through the html on the page and determined the information I was interested in was held in 'option' tags.

In [ ]:
clubList = []
for item in temp_r2.find_all('option'):
    #print(item.contents[0])
    temp_dict = {}
    if "Events" in item.contents[0]:
        #print(item)
        break
    elif "Participants" not in item.contents[0]:
        temp_dict['club_res_url'] = item.attrs['data-href']
        temp_dict['club_number'] = item.attrs['value']
        temp_dict['club_name'] = item.contents[0]
        clubList.append(temp_dict)
        ## used for dev/debug
        #print(item.attrs['data-href'])
        #print(item.attrs['value'])
        #print(item.contents[0])
        
print("number of clubs in the meet", len(clubList))
print("infomration about each club") 
clubList[0] 

The next step is to get the club results and retrieve the information about each swimmer

In [ ]:
r3_url = clubList[0]['club_res_url']
r3 = requests.get(r3_url)
temp_r3 = BeautifulSoup(r3.text, 'lxml')

After examining the HTML for the per club results page, the results of interest seem to be in 4 tables:
* table 0 - men's results
* table 1 - women's results
* table 2 - relay results
* table 3 - meet resource info

In [ ]:
temp_table = temp_r3.find_all("table")
len(temp_table)

In [ ]:
#look at the results for the men's table
count = 0
for item in temp_table[0].find_all('tr'):
    if item.has_attr('class'):
        print("row count", count, " ", item)
        print("new swimmer!!")
    #print(count)
    #if item.children[0].has_attr('class'):
    #    print("race time and dist available")
    #    print(item.children[0])
    elif count != 0:
        print("row count", count, len(item.contents))
        for subItem in item:
            print(len(subItem), subItem.string)
    else:
        print("row count", count)
    count += 1
    
    if count == 11:
        break

## Assess - www.swimming.ca

## Clean - www.swimming.ca